In [1]:
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

CLIENT_ID = os.getenv("MAL_CLIENT_ID")

In [3]:
url = "https://api.myanimelist.net/v2/anime/ranking"
API_FIELDS = ["mean","popularity","media_type","source","genres","rating"]

headers = {
    "X-MAL-CLIENT-ID": CLIENT_ID
}

params = {
    "ranking_type": "all",
    "limit": 500,
    "fields": ",".join(API_FIELDS)
}

In [4]:
response = requests.get(url, headers=headers, params=params)

print(response.status_code)

data = response.json() #check the status code
print(len(data["data"])) #check if we have the correct number of titles

200
500


In [14]:
result = []

DEMOGRAPHIC_NAMES = {"Shounen", "Seinen", "Shoujo", "Josei"}

for item in data['data']:
    node = item['node']

    genres_list = [genre["name"] for genre in node.get("genres", [])]

    genres = [genre for genre in genres_list if genre not in DEMOGRAPHIC_NAMES]
    demographic = [genre for genre in genres_list if genre in DEMOGRAPHIC_NAMES]
    #if not demographic:
        #demographic = ["Undefined"]

    result.append({
        "rank": item['ranking']['rank'],
        "score": node.get('mean'),
        "popularity": node.get('popularity'),
        "title": node['title'],
        "type": node.get('media_type'),
        "source": node.get('source'),
        "genres": ", ".join(genres),
        "demographic": ", ".join(demographic),
        "rating": node.get('rating')
    })

print(json.dumps(result[0], indent=2))

{
  "rank": 1,
  "score": 9.29,
  "popularity": 120,
  "title": "Sousou no Frieren",
  "type": "tv",
  "source": "manga",
  "genres": "Adventure, Drama, Fantasy",
  "demographic": "Shounen",
  "rating": "pg_13"
}


In [15]:
#turn the lst of dict into dataframe
COLUMNS = ["rank","score","popularity","title","type","source","genres","demographic","rating"]
df = pd.DataFrame(result, columns=COLUMNS)
df.head(10).style.hide(axis="index") #show data without the index

rank,score,popularity,title,type,source,genres,demographic,rating
1,9.290000,120,Sousou no Frieren,tv,manga,"Adventure, Drama, Fantasy",Shounen,pg_13
2,9.140000,830,Chainsaw Man Movie: Reze-hen,movie,manga,"Action, Fantasy, Gore, Urban Fantasy",Shounen,r
3,9.100000,3,Fullmetal Alchemist: Brotherhood,tv,manga,"Action, Adventure, Drama, Fantasy, Military",Shounen,r
4,9.070000,14,Steins;Gate,tv,visual_novel,"Drama, Psychological, Sci-Fi, Suspense, Time Travel",,pg_13
5,9.050000,21,Shingeki no Kyojin Season 3 Part 2,tv,manga,"Action, Drama, Gore, Military, Survival, Suspense",Shounen,r
6,9.050000,1512,Gintama: The Final,movie,manga,"Action, Comedy, Drama, Gag Humor, Historical, Parody, Samurai, Sci-Fi",Shounen,pg_13
7,9.050000,347,Gintama°,tv,manga,"Action, Comedy, Gag Humor, Historical, Parody, Samurai, Sci-Fi",Shounen,pg_13
8,9.040000,4670,Kingdom 6th Season,tv,manga,"Action, Historical, Military",Seinen,r
9,9.030000,8,Hunter x Hunter (2011),tv,manga,"Action, Adventure, Fantasy",Shounen,pg_13
10,9.020000,753,Ginga Eiyuu Densetsu,ova,novel,"Adult Cast, Drama, Military, Sci-Fi, Space",,r


In [16]:
#save to csv file
df.to_csv("anime_data.csv", index=False)